<a href="https://colab.research.google.com/github/rodrigosanchez900/AI/blob/main/buscador_de_passeios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para identificar 10 passeios. Os passeios devem acontecer hoje, amanhã ou depois de amanhã, necessariamente.
        Os passeios podem incluir cinema, teatro, circo, shows, exposições e atividades ao ar livre, como parques.
        Os passeios a serem sugeridos devem, se possível, acontecer nas proximidades da região indicada. Entretanto priorize a relevância.
        Os passeios devem ser muito relevantes.
        Se um tema tiver poucas notícias ou poucas reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Se o prompt conter uma atividade específica, como cinema, busque apenas filmes em cinema, e haja da mesma forma
        com teatro, circo, shows, exposições e atividades ao ar livre, como parques.
        Se o prompt conter um gênero, como comédia, busque apenas cinema, teatro, circo e shows de comédia.
        Identifique no máximo 10 passeios relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de passeios, especialista em atividades de lazer que incluem cinema, teatro, shows, circo, passeios ao ar livre,
        como parques, e atividades culturais em geral. Com base na lista de
        lançamentos mais recentes e relevantes do buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os passeios mais relevantes que poderíamos sugerir. As sugestões devem ocorrer hoje ou nos dois próximos dias subsequentes.
        O feedback deverá ser feito em formato de post sobre
        eles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher, com base nas suas pesquisas, os 5 passeios mais relevante entre eles, explicando sucintamente porque se destacaram,
        com informações relevantes como local,
        horários de início, por meio de um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais sobre passeios e atividades culturais para redes sociais.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final. Inclua emojis.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Sugestão de Passeios e Diversão 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o local (pode ser cidade, bairro ou CEP) para que eu possa sugerir bons passeios para hoje, amanhã e depois. Também pode indicar uma atividade específica, como cinema, ou o gênero, como comédia: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Sugestão de Passeios e Diversão 🚀
❓ Por favor, digite o local (pode ser cidade, bairro ou CEP) para que eu possa sugerir bons passeios para hoje, amanhã e depois. Também pode indicar uma atividade específica, como cinema, ou o gênero, como comédia: 01548-030 aventura
Maravilha! Vamos então criar o post sobre novidades em 01548-030 aventura

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para te ajudar a encontrar opções de passeios e aventuras em 01548-030 para os próximos dias, vou usar a ferramenta de busca para encontrar eventos e atividades relevantes.
> 
> 
> Com base nas minhas buscas, aqui estão algumas sugestões de atividades de aventura para você nos próximos dias, considerando a sua localização (01548-030):
> 
> 1.  **Parque Aventura da Figueira da Foz:** Localizado a uma curta distância de várias cidades, este parque oferece atividades de arborismo com diferentes níveis de dificuldade, como pontes, redes de cordas e slides. É uma ótima opção para famílias e grupos.
> 
> 2.  **Albufeira Adventure Park:** Próximo à praia de Santa Eulália, este parque oferece percursos de arborismo e paintball, sendo acessível a partir do centro da cidade.
> 
> 3.  **Pena Aventura:** Este parque oferece uma variedade de atividades lúdicas e de desporto aventura/natureza, incluindo Fantasticable, Salto Negativo, Slide, Escalada, Minigolfe, Paintball, Canyoning e muito mais.
> 
> 4.  **Timeout Aventura:** Oferece experiências de aventura e natureza, com atividades como Canyoning, Slide, Arborismo e Paintball. Também organiza team building, aniversários e programas para escolas.
> 
> 5.  **Adventure Park:** Parque de arborismo com atividades como orientação, canoagem e combate laser. Ideal para visitas de estudo e empresas.
> 
> Embora as buscas não tenham retornado atividades especificamente dentro do CEP 01548-030, estas opções são em locais acessíveis e oferecem diversas atividades de aventura para aproveitar o fim de semana.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Para planejar um post de qualidade sobre aventuras perto de 01548-030, preciso refinar minha busca e focar em opções mais próximas ou em atividades que possam ser realizadas na região ou acessadas facilmente a partir dela. Além disso, preciso verificar a disponibilidade e horários dessas atividades para os próximos dias.
> 
> Vou usar as seguintes buscas para obter informações mais específicas:
> 
> 
> Com base nas informações coletadas, aqui está um plano para um post sobre atividades de aventura perto de 01548-030 para os próximos dias.
> 
> **Título do Post:** Aventura Perto de Você: Descubra Emoções a Poucos Quilômetros de 01548-030!
> 
> **Assuntos a serem abordados:**
> 
> 1.  **Introdução:**
> 
>     *   Contextualizar a busca por atividades de aventura para moradores da região de 01548-030.
>     *   Mencionar a dificuldade de encontrar opções diretamente no CEP, mas apresentar alternativas acessíveis.
> 2.  **Opções de Parques de Aventura:**
> 
>     *   **Adventure Park:**
>         *   **Local:** Não especificado na busca inicial, mas é importante descobrir a localização exata para o post.
>         *   **Atividades:** Arborismo, orientação, canoagem, combate laser.
>         *   **Destaque:** Ideal para grupos, famílias, escolas e empresas. Possibilidade de festas de aniversário e team building.
>         *Apresentar a nova atividade Laser Tag
>     *   **Pena Aventura:**
>         *   **Local:** Não especificado na busca inicial, mas é importante descobrir a localização exata para o post.
>         *   **Atividades:** Fantasticable, Salto Negativo, Slide, Escalada, Paintball, Canyoning, entre outros.
>         *   **Destaque:** Grande variedade de atividades para todos os gostos e idades, incluindo opções aquáticas e terrestres.
>     *   **Diver Lanhoso:**
>         *   **Local:** Não especificado na busca inicial, mas é importante descobrir a localização exata para o post.
>         *   **Atividades:** Mais de 50 atividades de aventura (não especificadas na busca).
>         *   **Destaque:** Considerado o melhor parque de aventura da Península Ibérica, com opções de alojamento e restaurante panorâmico.
> 
> 3.  **Dicas Adicionais:**
> 
>     *   **Verificar a Necessidade de Agendamento:** Enfatizar a importância de verificar os horários de funcionamento e a necessidade de agendamento prévio, especialmente para grupos.
>     *   **Equipamentos e Vestimentas:** Orientar sobre os equipamentos de segurança fornecidos e as vestimentas adequadas para cada atividade.
>     *   **Condições Climáticas:** Lembrar de verificar as condições climáticas antes de ir e como isso pode afetar a disponibilidade de algumas atividades.
> 
> 4.  **Chamada para Ação:**
> 
>     *   Incentivar os leitores a explorarem as opções apresentadas e a planejarem sua próxima aventura.
>     *   Convidar os leitores a compartilharem suas experiências e sugestões nos comentários.
> 
> **Os 5 passeios mais relevantes:**
> 
> 1.  **Adventure Park:** Por oferecer uma variedade de atividades em um só lugar, ideal para grupos e famílias, e pela possibilidade de combinar atividades como arborismo, canoagem e combate laser.
> 2.  **Pena Aventura:** Pela diversidade de atividades, incluindo opções radicais como o Fantasticable e atividades aquáticas como o Canyoning, proporcionando experiências únicas para diferentes perfis de aventureiros.
> 3.  **Diver Lanhoso:** Por ser considerado o melhor parque de aventura da Península Ibérica, com mais de 50 atividades e opções de alojamento, oferecendo uma experiência completa de aventura e lazer.
> 4.  **Atividades ao ar livre em parques locais:** Explorar trilhas e áreas naturais em parques próximos a 01548-030 pode ser uma opção mais acessível e igualmente divertida para quem busca contato com a natureza e atividades físicas.
> 5.  **Eventos esportivos e radicais:** Ficar atento a eventos esportivos e radicais que possam ocorrer na região, como campeonatos de skate, escalada ou outras modalidades, pode ser uma forma de vivenciar a aventura de maneira diferente e conhecer novas pessoas com interesses em comum.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> 🧗‍♂️🌳 **Aventura Perto de Você: Descubra Emoções a Poucos Quilômetros de 01548-030!** 📍
> 
> Tá buscando adrenalina e diversão perto de casa? 🤔 Encontrar atividades de aventura no CEP 01548-030 pode ser um desafio, mas relaxa! 😉 Separamos opções incríveis a poucos quilômetros, perfeitas para você e sua galera! 🤩
> 
> 1️⃣ **Adventure Park:** 🌳
>    Arborismo, orientação, canoagem e o super divertido combate laser! 🔫 Ideal para famílias, escolas e empresas! Ah, e agora tem **Laser Tag**! 🥳
> 
> 2️⃣ **Pena Aventura:** ⛰️
>    Prepare-se para o Fantasticable, Salto Negativo, Slide, Escalada, Paintball e Canyoning! 💦 Muita aventura em um só lugar! 😉
> 
> 3️⃣ **Diver Lanhoso:** 🏞️
>    Considerado o melhor parque de aventura da Península Ibérica, com mais de 50 atividades! 😲 E ainda tem alojamento e restaurante panorâmico! 🍽️
> 
> ⚠️ **Dicas:**
> *   Confirme os horários e agende sua visita! 🗓️
> *   Use roupas adequadas e aproveite os equipamentos de segurança! 🦺
> *   Fique de olho no clima! ☀️🌧️
> 
> 👉 Marque seus amigos aventureiros e planeje sua próxima escapada! 🚀 Compartilhe suas experiências nos comentários! 👇
> 
> #AventuraPertoDeMim #Adrenalina #Diversao 
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais engajador e otimizado para o público do Instagram, com algumas pequenas alterações:
> 
> **Problemas:**
> 
> *   **Tom:** Embora amigável, o tom pode ser um pouco mais entusiasmado e jovial, com mais emojis e gírias leves (sem exagerar).
> *   **Chamada para Ação:** Pode ser mais direta e convidativa.
> *   **Hashtags:** Poderiam ser mais específicas e populares para aumentar o alcance.
> *   **Formatação:** O texto está um pouco denso; podemos quebrá-lo um pouco mais para facilitar a leitura.
> 
> **Sugestões:**
> 
> 1.  **Reescreva o início:**
>     *   "A FIM DE UMA AVENTURA MAS SEM GRANA PRA IR LONGE? 😱 Cola aqui que te mostro uns rolês iraaados pertinho do CEP 01548-030! 🤩"
> 
> 2.  **Adicione mais emojis:**
>     *   Use emojis mais variados e que representem cada atividade (ex: 🎯 para Paintball, 🧗 para escalada, etc.).
> 
> 3.  **Deixe a descrição das atividades mais chamativa:**
>     *   **Adventure Park:** "Arborismo, canoagem e LASER TAG pra você se sentir no Rambo! 💥 Diversão garantida pra geral! 👨‍👩‍👧‍👦"
>     *   **Pena Aventura:** "Se joga no Fantasticable, desafie a altura no Salto Negativo e detone no Paintball! 🎯 Preparado pra tanta adrenalina? 🔥"
>     *   **Diver Lanhoso:** "O MELHOR parque de aventura da Península Ibérica te espera com +50 atividades! 😲 E ainda dá pra relaxar no restaurante panorâmico! 🍕 Vista INCRÍVEL!"
> 
> 4.  **Mude a chamada para ação:**
>     *   "Marca a galera que topa um desses rolês M-A-N-D-A-T-Ó-R-I-O-S! 👇 E conta pra gente: qual dessas aventuras te deixou com mais vontade? 😉"
> 
> 5.  **Altere as hashtags:**
>     *   \#AventuraSP \#RoleEmSP \#AdrenalinaSP \#SaoPaulo \#FinalDeSemana
> 
> **Post revisado:**
> 
> 🧗‍♂️🌳 **A FIM DE UMA AVENTURA MAS SEM GRANA PRA IR LONGE? 😱 Cola aqui que te mostro uns rolês iraaados pertinho do CEP 01548-030! 🤩** 📍
> 
> Tá procurando adrenalina e diversão perto de casa? 🤔 Encontrar atividades de aventura no CEP 01548-030 pode ser um desafio, mas relaxa! 😉 Separamos opções INCRÍVEIS a poucos quilômetros, perfeitas pra você e sua galera! 🤩
> 
> 1️⃣ **Adventure Park:** 🌳
> Arborismo, canoagem e LASER TAG pra você se sentir no Rambo! 💥 Diversão garantida pra geral! 👨‍👩‍👧‍👦 Ah, e agora tem **Laser Tag**! 🥳
> 
> 2️⃣ **Pena Aventura:** ⛰️
> Se joga no Fantasticable, desafie a altura no Salto Negativo e detone no Paintball! 🎯 Preparado pra tanta adrenalina? 🔥 Muita aventura em um só lugar! 😉
> 
> 3️⃣ **Diver Lanhoso:** 🏞️
> O MELHOR parque de aventura da Península Ibérica te espera com +50 atividades! 😲 E ainda dá pra relaxar no restaurante panorâmico! 🍕 Vista INCRÍVEL! 🍽️
> 
> ⚠️ **Dicas:**
> 
> *   Confirme os horários e agende sua visita! 🗓️
> *   Use roupas adequadas e aproveite os equipamentos de segurança! 🦺
> *   Fique de olho no clima! ☀️🌧️
> 
> 👉 Marca a galera que topa um desses rolês M-A-N-D-A-T-Ó-R-I-O-S! 👇 E conta pra gente: qual dessas aventuras te deixou com mais vontade? 😉
> 
> \#AventuraSP \#RoleEmSP \#AdrenalinaSP \#SaoPaulo \#FinalDeSemana
> 


--------------------------------------------------------------
